In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from collections import Counter
import nltk
from nltk.util import ngrams

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
train_data = pd.read_csv("/content/drive/MyDrive/Dataset/Vashantor_CSV_Format/Train/Sylhet Train Translation.csv")
test_data = pd.read_csv("/content/drive/MyDrive/Dataset/Vashantor_CSV_Format/Test/Sylhet Test Translation.csv")
validation_data = pd.read_csv("/content/drive/MyDrive/Dataset/Vashantor_CSV_Format/Validation/Sylhet Validation Translation.csv")

In [4]:
train_data.head(5)

,bangla_speech,banglish_speech,sylhet_bangla_speech,sylhet_banglish_speech,region_name,english_speech
0,কেমন আছো ?,kemon acho?,ভালা আছনি?,vala achoni?,Sylhet,How are you?
1,আজকে আমার মন ভালো নেই,ajke amr mon valo nei,আইজকু আমার মন ভালা নায়,aijku amar mon vala nay,Sylhet,I'm not feeling well today
2,তুমি কি করো ?,tumi ki koro?,তুমি কিতা খরো?,tumi kita khoro?,Sylhet,what are you doing
3,এই গরমে আমার কিছু ভালো লাগে না,ei gorome amar kichu valo lage na,অউ গরমো আমার কুনতা ভালা লাগের না,ou gormo amar kunta vala lager na,Sylhet,I don't like anything this summer
4,ছেলেটি সাদা রঙয়ের একটি শার্ট পরে এসেছিল,cheleti sada ronger ekti shirt pore eshechilo,ফুয়াটায় এখটা সাদা রংগর শার্ট পিন্দিয়া আইছিল,fuyatay ekhta sada ronggor shirt pindiya aichilo,Sylhet,The boy came wearing a white shirt


In [5]:
train_data.tail(5)

,bangla_speech,banglish_speech,sylhet_bangla_speech,sylhet_banglish_speech,region_name,english_speech
1870,কাউকে মিথ্যা বলে খুশী করার থেকে সত্যি বলে কাঁদ...,kauke mittha bole khushi korar theke sotti bol...,কাউরে মিচা বলিয়া খুশির করার চেয়ে হাচা বলে কানদ...,kaure mica boliya khushir korar cheye hacha bo...,Sylhet,It's better to be happy for telling a lie than...
1871,"একে অপরের প্রতি যতো বেশী বিশ্বাস থাকবে, তাদের ...","eke oporer proti joto beshi bisshas thakbe, ta...","একে অপরের পতি যতো বেশি বিশ্বাস থাকবো, তাদের ভ...","eke oprer poti joto beshi bisshas thakbo, tad...",Sylhet,"The more trust they have in each other, the he..."
1872,বৃষ্টি বেড়ে তাপমাত্রা কমার সুখবর দিয়েছে আবহা...,brishti bere tapmatra komar sukhobor diyeche a...,মেগ বাড়িয়া তাপমাত্রা কমার সুখবর দিয়েছে আবহাওয়া...,meg bariya tapmatra komar sukhobor diyeche abh...,Sylhet,Meteorological office has given the good news ...
1873,পুরো বর্ষা মৌসুমে হাতে গোনা কয়েকদিন মাত্র বৃষ...,puro borsha moushume hate gona koyekdin matro ...,ফুরো মেগ মৌসুমো হতে গনা কয়েকদিন মাত্র মেগ অওয়া...,furo meg mousumo hote gona koyekdin matro meg ...,Sylhet,After only a handful of days of rain in the en...
1874,খাওয়ার অনেক আগেই যদি ভাত রান্না শেষ হয়ে গিয়ে থ...,khaoyar onek agei jodi vat ranna shesh hoye gi...,খাওয়ার অনেক আগেই যদি ভাত রান্দা শেষ হয়ে অইয়া গ...,khaoyar onek agei jodi vat randa shesh hoye oi...,Sylhet,If the rice is done cooking long before eating...


In [6]:
test_data.head(5)

,bangla_speech,banglish_speech,sylhet_bangla_speech,sylhet_banglish_speech,region_name,english_speech
0,তোমার আব্বু কেমন আছে?,tomar abbu kemon ache?,তোমার আব্বা বালা আছইন নি?,tomar abba bala achoin ni?,Sylhet,How is your father?
1,আমার বড় বোনের আজকে মন ভালো নেই,amar boro boner ajke mon valo nei,আমার বড় বইনর আইজ মন ভালা নায়,amar boro boinor aij mon vala nay,Sylhet,My elder sister is not feeling well today
2,তুমি কি আমাকে এই কাজটি করে দিতে পারবে?,tumi ki amake ei kajti kore dite parbe?,তুমি কিতা মোরে এই কাজটা করাই দিতা পারবা নি?,tumi kita more ei kajta korai dita parba ni?,Sylhet,Can you do this for me?
3,এই গরমে বাহিরে বের হতে ইচ্ছা করে না,ei gorome bahire ber hote iccha kore na,এই গরমো বাইরে বাইর অইতে জানে কয় না,ei gormo baire bair oite jane koy na,Sylhet,Don't want to go out in this summer
4,তোমার সব কথাই ভুল,tomar sob kothai vul,তোমার সব মাত ভুল,tomar sob mat vul,Sylhet,Everything you say is wrong


In [7]:
test_data.tail(5)

,bangla_speech,banglish_speech,sylhet_bangla_speech,sylhet_banglish_speech,region_name,english_speech
370,তুমি কি আমাকে এক গ্লাস পানি এনে দিতে পারবে ?,tumi ki amake ek glass pani ene dite parbe ?,তুমি কি আমারে এক গ্লাস ফানি এনে দিতায় ফারবা?,tumi ki amare ek glass fani ene ditay farba?,Sylhet,Can you get me a glass of water?
371,আমি পারবো না পানি দিতে,ami parbo na pani dite,আমি পারবো না ফানি দিতে,ami parbo na fani dite,Sylhet,I can't give water
372,তোমার ভাই পড়ালেখাতে অনেক মেধাবি,tomar vai poralekhate onek medhabi,তোমার ভাই ফড়ালেখাতে বহুত মেধাবি,tomar vai foralekhate bohut medhabi,Sylhet,Your brother is very talented in studies
373,আচ্ছা বলো দেখি বাংলাদেশে কয়টি জেলা?,accha bolo dekhi bangladeshe koyti jela?,আইচ্চা কও দেখি বাঙলাদেশো কয়টা জেলা?,aiccha kow dekhi bangladesho koyta jela?,Sylhet,Well tell me how many districts in Bangladesh?
374,সামনের দিকে যেয়ে মেয়েটি অনেক হাসবে,samner dike jeye meyeti onek hashbe,সামনর দিকে যাইয়া ফুড়িটা বহুত হাসবো,samnor dike jaiya furita bohut hashbo,Sylhet,The girl will smile a lot going forward


In [8]:
validation_data.head(5)

,bangla_speech,banglish_speech,sylhet_bangla_speech,sylhet_banglish_speech,region_name,english_speech
0,বাংলাদেশে ৬৪ টা জেলা,bangladeshe ৬৪ ta jela,বাংলাদেশো ৬৪ টা জেলা,bangladesho ৬৪ ta jela,Sylhet,64 districts in Bangladesh
1,আমরা সবাই গতকাল বাহিরে গিয়েছিলাম,amra sobai gotokal bahire giyechilam,আমরা হকল গতকাইল বাহিরো গিয়েছিলাম,amra hokol gotokail bahiro giyechilam,Sylhet,We all went out yesterday
2,তোমার কথা বলার ধরন অনেক সুন্দর,tomar kotha bolar dhoron onek sundor,তোমার মাত বলার ধরন বহুত সুন্দর,tomar mat bolar dhoron bohut shundor,Sylhet,Your way of speaking is very nice
3,বরিশালের মানুষ কেমন হয়?,barishaler manush kemon hoy?,বরিশালর মানুষ কিলা অইন?,barishalor manush kila oin?,Sylhet,How are the people of Barisal?
4,খুলনা জেলা কি অনেক সুন্দর?,khulna jela ki onek sundor?,খুলনা জেলা কিতা বহুত সুন্দর?,khulna jela kita bohut shundor?,Sylhet,Khulna district is very beautiful?


In [9]:
validation_data.tail(5)

,bangla_speech,banglish_speech,sylhet_bangla_speech,sylhet_banglish_speech,region_name,english_speech
245,তুমি যাও এখান থেকে,tumi jao ekhan theke,তুমিতাইন হরো ওখান তাকি,tumitain horo okhan taki,Sylhet,you go from here
246,আপনি যান এখান থেকে,apni jan ekhan theke,আফনে হরোউকা ওখান তাকি,afne horouka okhan taki,Sylhet,You go from here
247,তুই যা এখান থেকে,tui ja ekhan theke,তুই হর ওখান তাকি,tui hor okhan taki,Sylhet,you go from here
248,আপনারা পার্কে ঘুরে বেড়ান,apnara parke ghure beran,আফনারা পার্ক-ও ঘুরিয়া বেড়াউকা,afnara park-o ghuriya berauka,Sylhet,You go for a walk in the park
249,উনি চেষ্টা করেন,uni cheshta koren,তাইন চেষ্টা করোইন,tain cheshta koroin,Sylhet,He tries


In [10]:
!pip install transformers torch pandas

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [11]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 3.0 MB/s eta 0:00:00


In [12]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=8372521bfd3b9d06d3d42610c01ca43136a0c91988d265143c45f32da0bee7df
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [13]:
!pip install sentencepiece

In [14]:
!pip install transformers[sentencepiece]

In [15]:
!transformers-cli cache clear

2024-05-24 08:17:07.193375: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-24 08:17:07.193432: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-24 08:17:07.194782: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-24 08:17:07.202060: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-24 08:17:08.222906: W tensorflow/compiler/tf2

In [16]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 4.6 MB/s eta 0:00:00


In [17]:
!pip install accelerate -U

In [18]:
!pip install git+https://github.com/csebuetnlp/normalizer

  Cloning https://github.com/csebuetnlp/normalizer to /tmp/pip-req-build-0w80ikk_
  Running command git clone --filter=blob:none --quiet https://github.com/csebuetnlp/normalizer /tmp/pip-req-build-0w80ikk_
  Resolved https://github.com/csebuetnlp/normalizer to commit d405944dde5ceeacb7c2fd3245ae2a9dea5f35c9
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.0/185.0 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for normalizer: filename=normalizer-0.0.1-py3-none-any.whl size=6859 sha256=fb0156f9f47c83faa0c6069ec07c088f3b72bf92b0b915f07359bcf5b5e8d88e
  Stored in directory: /tmp/pip-ephem-wheel-cache-v5tsp5wf/wheels/2e/79/9c/cd96d490298305d51d2da11484bb2c25fd1f759a6906708282
  Created wheel for emoji: filename=emoji-1.4.2-py3-none-any.whl size=186459 sha256=634edd2f834450755a865ccfe7ca5f2224ec5f0

In [19]:
!pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [ ]:
#!pip install transformers==4.10.3

In [20]:
!pip install torch transformers

In [ ]:
#!pip install accelerate==0.20.3

In [21]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, DataCollatorForSeq2Seq, Trainer, TrainingArguments
from normalizer import normalize
import pandas as pd
import numpy as np
import nltk
from nltk.translate.bleu_score import corpus_bleu
from sacrebleu import sentence_bleu
from rouge_score import rouge_scorer
from sklearn.model_selection import train_test_split
import os


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [22]:
train_data = pd.read_csv("/content/drive/MyDrive/Dataset/Vashantor_CSV_Format/Train/Sylhet Train Translation.csv")
test_data = pd.read_csv("/content/drive/MyDrive/Dataset/Vashantor_CSV_Format/Test/Sylhet Test Translation.csv")
validation_data = pd.read_csv("/content/drive/MyDrive/Dataset/Vashantor_CSV_Format/Validation/Sylhet Validation Translation.csv")

In [23]:
train_data.head()

,bangla_speech,banglish_speech,sylhet_bangla_speech,sylhet_banglish_speech,region_name,english_speech
0,কেমন আছো ?,kemon acho?,ভালা আছনি?,vala achoni?,Sylhet,How are you?
1,আজকে আমার মন ভালো নেই,ajke amr mon valo nei,আইজকু আমার মন ভালা নায়,aijku amar mon vala nay,Sylhet,I'm not feeling well today
2,তুমি কি করো ?,tumi ki koro?,তুমি কিতা খরো?,tumi kita khoro?,Sylhet,what are you doing
3,এই গরমে আমার কিছু ভালো লাগে না,ei gorome amar kichu valo lage na,অউ গরমো আমার কুনতা ভালা লাগের না,ou gormo amar kunta vala lager na,Sylhet,I don't like anything this summer
4,ছেলেটি সাদা রঙয়ের একটি শার্ট পরে এসেছিল,cheleti sada ronger ekti shirt pore eshechilo,ফুয়াটায় এখটা সাদা রংগর শার্ট পিন্দিয়া আইছিল,fuyatay ekhta sada ronggor shirt pindiya aichilo,Sylhet,The boy came wearing a white shirt


In [24]:
train_data = train_data.drop(columns=['banglish_speech ','sylhet_banglish_speech ','region_name ','english_speech'],axis=1)
train_data.head()

,bangla_speech,sylhet_bangla_speech
0,কেমন আছো ?,ভালা আছনি?
1,আজকে আমার মন ভালো নেই,আইজকু আমার মন ভালা নায়
2,তুমি কি করো ?,তুমি কিতা খরো?
3,এই গরমে আমার কিছু ভালো লাগে না,অউ গরমো আমার কুনতা ভালা লাগের না
4,ছেলেটি সাদা রঙয়ের একটি শার্ট পরে এসেছিল,ফুয়াটায় এখটা সাদা রংগর শার্ট পিন্দিয়া আইছিল


In [25]:
test_data = test_data.drop(columns=['banglish_speech ','sylhet_banglish_speech ','region_name ','english_speech'],axis=1)
test_data.head()

,bangla_speech,sylhet_bangla_speech
0,তোমার আব্বু কেমন আছে?,তোমার আব্বা বালা আছইন নি?
1,আমার বড় বোনের আজকে মন ভালো নেই,আমার বড় বইনর আইজ মন ভালা নায়
2,তুমি কি আমাকে এই কাজটি করে দিতে পারবে?,তুমি কিতা মোরে এই কাজটা করাই দিতা পারবা নি?
3,এই গরমে বাহিরে বের হতে ইচ্ছা করে না,এই গরমো বাইরে বাইর অইতে জানে কয় না
4,তোমার সব কথাই ভুল,তোমার সব মাত ভুল


In [26]:
train_data.rename(columns={'bangla_speech ': 'input_text', 'sylhet_bangla_speech '	: 'labels'}, inplace=True)
train_data.head()

,input_text,labels
0,কেমন আছো ?,ভালা আছনি?
1,আজকে আমার মন ভালো নেই,আইজকু আমার মন ভালা নায়
2,তুমি কি করো ?,তুমি কিতা খরো?
3,এই গরমে আমার কিছু ভালো লাগে না,অউ গরমো আমার কুনতা ভালা লাগের না
4,ছেলেটি সাদা রঙয়ের একটি শার্ট পরে এসেছিল,ফুয়াটায় এখটা সাদা রংগর শার্ট পিন্দিয়া আইছিল


In [27]:
# Rename the columns to match the expected format
test_data.rename(columns={'bangla_speech ': 'input_text', 'sylhet_bangla_speech '	: 'labels'}, inplace=True)
test_data.head()

,input_text,labels
0,তোমার আব্বু কেমন আছে?,তোমার আব্বা বালা আছইন নি?
1,আমার বড় বোনের আজকে মন ভালো নেই,আমার বড় বইনর আইজ মন ভালা নায়
2,তুমি কি আমাকে এই কাজটি করে দিতে পারবে?,তুমি কিতা মোরে এই কাজটা করাই দিতা পারবা নি?
3,এই গরমে বাহিরে বের হতে ইচ্ছা করে না,এই গরমো বাইরে বাইর অইতে জানে কয় না
4,তোমার সব কথাই ভুল,তোমার সব মাত ভুল


In [28]:
validation_data.head()

,bangla_speech,banglish_speech,sylhet_bangla_speech,sylhet_banglish_speech,region_name,english_speech
0,বাংলাদেশে ৬৪ টা জেলা,bangladeshe ৬৪ ta jela,বাংলাদেশো ৬৪ টা জেলা,bangladesho ৬৪ ta jela,Sylhet,64 districts in Bangladesh
1,আমরা সবাই গতকাল বাহিরে গিয়েছিলাম,amra sobai gotokal bahire giyechilam,আমরা হকল গতকাইল বাহিরো গিয়েছিলাম,amra hokol gotokail bahiro giyechilam,Sylhet,We all went out yesterday
2,তোমার কথা বলার ধরন অনেক সুন্দর,tomar kotha bolar dhoron onek sundor,তোমার মাত বলার ধরন বহুত সুন্দর,tomar mat bolar dhoron bohut shundor,Sylhet,Your way of speaking is very nice
3,বরিশালের মানুষ কেমন হয়?,barishaler manush kemon hoy?,বরিশালর মানুষ কিলা অইন?,barishalor manush kila oin?,Sylhet,How are the people of Barisal?
4,খুলনা জেলা কি অনেক সুন্দর?,khulna jela ki onek sundor?,খুলনা জেলা কিতা বহুত সুন্দর?,khulna jela kita bohut shundor?,Sylhet,Khulna district is very beautiful?


In [29]:
validation_data = validation_data.drop(columns=['banglish_speech ','sylhet_banglish_speech ','region_name ','english_speech'],axis=1)
validation_data.head()

,bangla_speech,sylhet_bangla_speech
0,বাংলাদেশে ৬৪ টা জেলা,বাংলাদেশো ৬৪ টা জেলা
1,আমরা সবাই গতকাল বাহিরে গিয়েছিলাম,আমরা হকল গতকাইল বাহিরো গিয়েছিলাম
2,তোমার কথা বলার ধরন অনেক সুন্দর,তোমার মাত বলার ধরন বহুত সুন্দর
3,বরিশালের মানুষ কেমন হয়?,বরিশালর মানুষ কিলা অইন?
4,খুলনা জেলা কি অনেক সুন্দর?,খুলনা জেলা কিতা বহুত সুন্দর?


In [30]:
# Rename the columns to match the expected format
validation_data.rename(columns={'bangla_speech ': 'input_text', 'sylhet_bangla_speech '	: 'labels'}, inplace=True)
validation_data.head()

,input_text,labels
0,বাংলাদেশে ৬৪ টা জেলা,বাংলাদেশো ৬৪ টা জেলা
1,আমরা সবাই গতকাল বাহিরে গিয়েছিলাম,আমরা হকল গতকাইল বাহিরো গিয়েছিলাম
2,তোমার কথা বলার ধরন অনেক সুন্দর,তোমার মাত বলার ধরন বহুত সুন্দর
3,বরিশালের মানুষ কেমন হয়?,বরিশালর মানুষ কিলা অইন?
4,খুলনা জেলা কি অনেক সুন্দর?,খুলনা জেলা কিতা বহুত সুন্দর?


In [31]:
import torch
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

model_name = "facebook/mbart-large-50"
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50", src_lang="en_XX", tgt_lang="ro_RO")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/531 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

In [32]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

In [ ]:
# Load the saved model
model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/Movies/Sylhet_mBart50_model.pt")

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Movies/Sylhet_mBart50_tokenizer.json")

In [33]:
import torch
from normalizer import normalize
from transformers import MT5ForConditionalGeneration, AutoTokenizer ,DataCollatorForSeq2Seq, Trainer, TrainingArguments
import os

In [ ]:
""""model_name = "csebuetnlp/banglat5"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.83k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [34]:
from torch.utils.data import Dataset, DataLoader

In [35]:
class Seq2SeqDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=128):
        """
        Args:
            data: A DataFrame containing 'input_text' and 'labels' columns.
            tokenizer: A Hugging Face tokenizer.
            max_length: Maximum sequence length.
        """
        self.input_text = data['input_text'].apply(normalize).tolist()
        self.labels = data['labels'].apply(normalize).tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.input_text)

    def __getitem__(self, idx):
        input_text = self.input_text[idx]
        label_text = self.labels[idx]

        # Tokenize the input text
        input_encodings = self.tokenizer(
            input_text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        # Tokenize the label text to get its 'input_ids' and 'attention_mask'
        label_encodings = self.tokenizer(
            label_text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        return {
            'input_ids': input_encodings['input_ids'].squeeze(),
            'attention_mask': input_encodings['attention_mask'].squeeze(),
            'labels': label_encodings['input_ids'].squeeze(),
        }


In [36]:
# Create train , test and validation datasets
train_dataset = Seq2SeqDataset(train_data, tokenizer)
test_dataset = Seq2SeqDataset(test_data, tokenizer)
validation_dataset = Seq2SeqDataset(validation_data, tokenizer)

# Create train , test and validation dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)  #batch_size=32
test_dataloader = DataLoader(test_dataset, batch_size=16) #batch_size=32
validation_dataloader = DataLoader(validation_dataset, batch_size=16) #batch_size=32


# **For training dataset**

In [ ]:
# Assuming you have already created the train_dataloader
for data_point in train_dataloader:
    input_ids = data_point['input_ids']
    attention_mask = data_point['attention_mask']
    labels = data_point['labels']
''' print("Input IDs:", input_ids)
    print("Attention Mask:", attention_mask)
    print("Labels:", labels)
'''

' print("Input IDs:", input_ids)\n    print("Attention Mask:", attention_mask)\n    print("Labels:", labels)\n'

# **For test dataset**

In [ ]:
# Assuming you have already created the test_dataloader
for data_point in test_dataloader:
    input_ids = data_point['input_ids']
    attention_mask = data_point['attention_mask']
    labels = data_point['labels']
''' print("Input IDs:", input_ids)
    print("Attention Mask:", attention_mask)
    print("Labels:", labels)
'''

' print("Input IDs:", input_ids)\n    print("Attention Mask:", attention_mask)\n    print("Labels:", labels)\n'

# **For validation dataset**

In [ ]:
# Assuming you have already created the validation_dataloader
for data_point in validation_dataloader:
    input_ids = data_point['input_ids']
    attention_mask = data_point['attention_mask']
    labels = data_point['labels']
'''    print("Input IDs:", input_ids)
    print("Attention Mask:", attention_mask)
    print("Labels:", labels)'''

'    print("Input IDs:", input_ids)\n    print("Attention Mask:", attention_mask)\n    print("Labels:", labels)'

# **from training dataset , I have done only one example decoding**

In [ ]:
# Assuming you have already created the train_dataloader with shuffle
for data_point in train_dataloader:
    input_ids = data_point['input_ids'][0]  # Assuming you want the first example
    attention_mask = data_point['attention_mask'][0]
    labels = data_point['labels'][0]

    # Decode input_ids and labels
    input_text = tokenizer.decode(input_ids, skip_special_tokens=True)
    output_text = tokenizer.decode(labels, skip_special_tokens=True)

    print("Input Text:", input_text)
    print("Output Text:", output_text)

    # Exit the loop after processing the first example
    break


Input Text: তোমরা ভালো আছ জেনে খুশি হলাম
Output Text: তোমরা ভালো আছ জানি খুশি অইলাম


# **from testing dataset , I have done only one example decoding**

In [ ]:
# Assuming you have already created the test_dataloader
for data_point in test_dataloader:
    input_ids = data_point['input_ids'][0]  # Assuming you want the first example
    attention_mask = data_point['attention_mask'][0]
    labels = data_point['labels'][0]

    # Decode input_ids and labels
    input_text = tokenizer.decode(input_ids, skip_special_tokens=True)
    output_text = tokenizer.decode(labels, skip_special_tokens=True)

    print("Input Text:", input_text)
    print("Output Text:", output_text)

    # Exit the loop after processing the first example
    break


Input Text: তোমার আব্বু কেমন আছে?
Output Text: তোর আব্বা কেক্কা আছে?


# **from validation dataset , I have done only one example decoding**

In [ ]:
# Assuming you have already created the validation_dataloader
for data_point in validation_dataloader:
    input_ids = data_point['input_ids'][0]  # Assuming you want the first example
    attention_mask = data_point['attention_mask'][0]
    labels = data_point['labels'][0]

    # Decode input_ids and labels
    input_text = tokenizer.decode(input_ids, skip_special_tokens=True)
    output_text = tokenizer.decode(labels, skip_special_tokens=True)

    print("Input Text:", input_text)
    print("Output Text:", output_text)

    # Exit the loop after processing the first example
    break


Input Text: বাংলাদেশে ৬৪ টা জেলা
Output Text: বাংলাদেশে ৬৪ গা জেলা


In [37]:
# Move the model to the device (CPU or GPU)
model.to(device)

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): Embedding(250054, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): MBartScaledWordEmbedding(250054, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (fi

In [ ]:
#pip install transformers==4.30


In [42]:
from transformers import TrainingArguments
from transformers import AdamW

from torch.optim import AdamW

# Create a custom optimizer using torch.optim.AdamW
custom_optimizer = AdamW(
    model.parameters(),
    lr=1e-3,  # Learning rate
    eps=1e-8,  # Epsilon value to prevent division by zero
    weight_decay=0.01,  # Weight decay (L2 regularization)
)

#if you have 1,000 training examples and a batch size of 100, you would have 10 iterations in each epoch (1,000 / 100 = 10)
'''
This parameter determines how many small batches are accumulated before performing a weight update.
In your code, it's set to 8, which means you'll accumulate gradients over 8 small batches before performing a weight update.
This effectively simulates a larger batch size without requiring more GPU memory.
So, you are updating the model's weights less frequently compared to the number of actual batches processed.
'''



'''
Learning rate determines how quickly the model learns from the data.
The learning rate scheduler type is set to "cosine_with_restarts," which is a type of learning rate schedule.
 Warmup steps are the number of initial training steps with a smaller learning rate, and weight decay introduces L2 regularization to the optimizer.
'''



'''
Number of Iterations per Epoch = Number of Training Samples / Batch Size
Total Iterations = Number of Iterations per Epoch * Number of Epochs
'''
# Define the TrainingArguments
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/Movies',
    num_train_epochs=50,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    evaluation_strategy="epoch",
    save_strategy="no",  # Align save strategy with evaluation strategy
    ##save_total_limit=1,
    #save_steps=700,
    learning_rate=1e-3,
    do_train=True,
    do_eval=True,
    remove_unused_columns=False,
    push_to_hub=False,
    report_to="none",
    #load_best_model_at_end=True,
    lr_scheduler_type="cosine_with_restarts",
    weight_decay=0.01,
    logging_dir='/content/drive/MyDrive/Movies',
    logging_steps=10,  # Log every 10 steps,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [43]:
from transformers import DataCollatorForSeq2Seq

# Create a data collator for sequence-to-sequence tasks
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,  # Your Hugging Face tokenizer
    model=model,
    padding=True,
    max_length=128,
    label_pad_token_id=tokenizer.pad_token_id,
)


In [44]:

# Define the Trainer with the custom optimizer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    optimizers=(custom_optimizer, None),  # Pass the custom optimizer here
)


# **Training start here**

In [45]:
# Fine-tune the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2717: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


OutOfMemoryError: CUDA out of memory. Tried to allocate 978.00 MiB. GPU 

# **Saving model and tokenizer**

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Save the model
model.save_pretrained("/content/drive/MyDrive/Movies/Sylhet_mBart50_model.pt")

# Save the tokenizer
tokenizer.save_pretrained("/content/drive/MyDrive/Movies/Sylhet_mBart50_tokenizer.json")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5, 'forced_eos_token_id': 2}


('/content/drive/MyDrive/Movies/Sylhet_mBart50_tokenizer.json/tokenizer_config.json',
 '/content/drive/MyDrive/Movies/Sylhet_mBart50_tokenizer.json/special_tokens_map.json',
 '/content/drive/MyDrive/Movies/Sylhet_mBart50_tokenizer.json/sentencepiece.bpe.model',
 '/content/drive/MyDrive/Movies/Sylhet_mBart50_tokenizer.json/added_tokens.json',
 '/content/drive/MyDrive/Movies/Sylhet_mBart50_tokenizer.json/tokenizer.json')

# **Loading trained model and tokenizer**

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load the saved model
model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/Movies/Noakhali_model.pt")

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Movies/Noakhali_tokenizer.json")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


In [ ]:
!pip install datasets

In [ ]:
!pip install python-Levenshtein

In [ ]:
!pip install jiwer

In [ ]:
# Move the model to the device (CPU or GPU)
model.to(device)

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): Embedding(250054, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): Embedding(250054, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm)

In [ ]:
!pip install rouge-score
#https://github.com/google-research/google-research/tree/master/rouge
#https://huggingface.co/spaces/evaluate-metric/rouge [Different types of ROUGE scores]

In [ ]:
!pip install evaluate

# **Loading evaluation metrics**

In [ ]:
import torch
import Levenshtein
from evaluate import load
# Define the move_to_device function
def move_to_device(batch, device):
    if isinstance(batch, torch.Tensor):
        return batch.to(device)
    elif isinstance(batch, list):
        return [move_to_device(item, device) for item in batch]
    elif isinstance(batch, dict):
        return {key: move_to_device(value, device) for key, value in batch.items()}
    else:
        return batch  # If it's not a tensor, list, or dict, leave it as is

# Load the evaluation metric for Character Error Rate (CER) and Word Error Rate (WER) and Exact Match(em)
cer_metric = load("cer")
wer_metric = load("wer")
meteor = load('meteor')
exact_match_metric = load("exact_match")

# Load BLEU and ROUGE metrics
bleu_metric = load("bleu")
rouge_metric = load('rouge')

# Initialize lists to store generated translations and references
generated_translations = []
references = []

# Generate translations for the test dataset
for batch in test_dataloader:
    # Move the batch to CUDA
    batch = move_to_device(batch, 'cuda')

    input_text = batch['input_ids']  # Access the input_text using the correct key
    labels = batch['labels']  # Access the labels using the correct key

    # Generate translations
    translation_ids = model.generate(input_text, max_length=512, num_beams=4, length_penalty=2.0, early_stopping=True)

    # Move the translation_ids to CPU to decode
    translation_ids = translation_ids.to('cpu')

    generated_translation = tokenizer.batch_decode(translation_ids, skip_special_tokens=True)

    generated_translations.extend(generated_translation)
    references.extend(tokenizer.batch_decode(labels, skip_special_tokens=True))  # Decoding the label IDs

# Make sure to move generated_translations back to CPU for evaluation if necessary
generated_translations = [translation if not isinstance(translation, str) else translation for translation in generated_translations]



[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
print("Number of generated translations:", len(generated_translations))
print("Number of references:", len(references))

In [ ]:
print(generated_translations)

['তোমারের আছে কেমন কিল্লেই', 'আর বড় বোনের আজরে মন ভালা নাই', 'তুউই কি আরে এই কামাই করে না', 'এর তুন এই গরমে বহুত ইচ্ছু করে না', 'তোয়ার ফ্রিয় কতাই', 'তোমার মত খারাপ থাই আই আর এককানও দেন', 'বিয়ার লাই আর লাইগছে হরিবারে লই মইধ্যে', 'আব্বা আরে গতরে গতকাল এককান লই দেয়া করতো কইছিলো', 'হোলাগা হোলাগা আমার একদমই ভালা লাগছে', 'মাইয়াগা গোলা রঙের শাড়ি রইছে', 'আর দুইদিন পরে বিয়া অইছে', 'হোলাগার এখন আর বিয়ার খাইতে বন্দুগতে হারে না', 'মেয়েটি সকালে ঘুম থেকে উঠেই হড়াই গেছে', 'আর চাচা বিদেশে তুন গতকাল বাংলাদেশে আইছে', 'আর বাসার দারানের আবেগ ভালা মানুষ', 'আর বড্ডা খুলনা ঘুইরতে গেছে', 'আর ঘুইরতে অনেক ভালা লাগে', 'আই বিলাইগুন চাই', 'তুউই আরে বিরক্ত করো না', 'হেতনে আরে অনেক অপমান করেছে', 'আর সকালে উডিতে বই ভালা লাগে', 'আর ছোড ভাই একদমতে চায় না', 'আর ছোটোট বড্ডা ভাই স্কুলে যাইতাম না', 'আর বাপের ভাইয়ের হরালেয়া কইরতাম ভালা লাগে না', 'আই ওকগা মার মাডে ঘরে গেছে', 'আন্নে কি আরে কোনদিনও ভালোবাইসতে?', 'আর বন্ধু আর বন্ধু আজরে সকালে', 'তুমি আর মনে অই বুঝো অইঝো না', 'আর আব্বাু অনেক ভালা রান্না করে', 'আর হ

In [ ]:
print(references)

['তোর আব্বা কেক্কা আছে?', 'আর বড় আফার আইজ্জা মন ভালা নাই', 'তুই কি আরে এই কাম আন করি দিতা হাইরবা নি?', 'এই গরমেত্তে বাইর অইত ইচ্ছা করে না', 'তোর বেক কতাই ভুল', 'তোর মতো শয়তান হোলা আই এইগগাও দেই নো', 'বিয়ার লাই আর আম্মা আর লাই হোলা টোগার', 'কাইল্গা আব্বা আরে এক হোলার লগে দেয়া কইরতে কইছিলো', 'হোলাগারে আর এক্কানাও ভালা লাগে নো', 'মাইয়া গা গোলাপি রঙের কার হিনছিলো', 'দুইদিন হরে আর বিয়া ওইবো', 'হোলাগা অন আর বিয়ালে খেলতো বার ওনা', 'মাইয়া গা সকালে উঠিই হইরতো বইয়ে', 'কাইল্গা আর চাচা বিদাশ তুন বাংলাদেশে আইছে', 'আর বাসার দারোয়ান মেলা ভালা মানুষ', 'আর বন্ধু খুলনা ঘুইত্তো গেছে', 'আর ঘুইত্তে মেলা ভালা লাগে', 'আই বিদেশ যাই ঘুরত্তাম চাই', 'তুই আরে জালাইয়ো নাহ তো', 'হেতন আরে মেলা অপমান কইচ্ছে', 'আর বেনে উঠি বই হইরতে মেলা ভাল্লাগে', 'আর ভাই ছোডগা এক্কারেই হড়ালেয়া কইরতো চায় না', 'আর ভাই ছোডগা ইস্কুলে যাইতো চায় নাহ', 'আর ছোড ভাইয়ের ইস্কুলে যাই হরালেয়া করতে চায় না', 'আই একদিন ইস্কুলের মাঠে মাতা ঘুরি হরি গেছিলাম', 'আন্নে কি আরে কোনদিন ভালা বাইসতেন নো?', 'আর বন্ধু আইজকা ফুটবল খেলছে', 'তুই আ

In [ ]:
# Calculate Character Error Rate (CER) and Word Error Rate (WER)
results_CER = cer_metric.compute(predictions=generated_translations, references=references)
results_WER = wer_metric.compute(predictions=generated_translations, references=references)

# Calculate Exact Match (EM) and METEOR(M)
results_em = exact_match_metric.compute(predictions=generated_translations, references=references)
results_met = meteor.compute(predictions=generated_translations, references=references)

# Calculate Bilingual Evaluation Understudy (BLEU) and Recall-Oriented Understudy for Gisting Evaluation (ROUGE)
results_bleu = bleu_metric.compute(predictions=generated_translations, references=references)
results_rouge = rouge_metric.compute(predictions=generated_translations, references=references)


# Calculate Levenshtein Distance
levenshtein_distances = [Levenshtein.distance(generated, reference) for generated, reference in zip(generated_translations, references)]


# **Printing every evaluation metrics**

In [ ]:
print(results_CER)
print(results_WER)
print(results_em)
print(results_met)
print(results_bleu)
print(results_rouge)
print(levenshtein_distances)

0.7605527292726523
1.0138732658417697
{'exact_match': 0.0}
{'meteor': 0.0476434539250976}
{'bleu': 0.0, 'precisions': [0.08479874951152794, 0.008241758241758242, 0.000552791597567717, 0.0], 'brevity_penalty': 0.9486124107322798, 'length_ratio': 0.9498886414253898, 'translation_length': 2559, 'reference_length': 2694}
{'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, 'rougeLsum': 0.0}
[20, 25, 26, 34, 21, 27, 23, 36, 23, 21, 33, 25, 29, 32, 23, 24, 17, 29, 15, 11, 25, 28, 32, 44, 38, 31, 22, 17, 18, 25, 19, 29, 29, 40, 15, 36, 18, 18, 27, 23, 18, 38, 38, 36, 34, 18, 34, 24, 44, 28, 29, 33, 25, 23, 51, 25, 22, 35, 23, 37, 38, 33, 34, 17, 32, 26, 30, 43, 37, 32, 31, 31, 31, 30, 35, 35, 34, 38, 33, 27, 22, 20, 22, 23, 19, 26, 38, 37, 37, 35, 21, 55, 39, 40, 35, 37, 31, 29, 29, 31, 28, 21, 32, 38, 36, 33, 36, 47, 35, 35, 43, 34, 36, 31, 23, 28, 38, 33, 15, 28, 32, 22, 28, 36, 32, 24, 35, 29, 38, 32, 18, 31, 22, 29, 35, 33, 56, 39, 22, 31, 35, 28, 24, 40, 42, 30, 32, 33, 26, 17, 38, 40, 34, 38, 

In [ ]:
total_correct = 0
total_samples = len(references)

for generated, reference in zip(generated_translations, references):
    levenshtein_distance = Levenshtein.distance(generated, reference)
    max_length = max(len(generated), len(reference))
    accuracy = 1 - (levenshtein_distance / max_length)
    if accuracy >= 0.8:  # Adjust the threshold as needed
        total_correct += 1

accuracy = total_correct / total_samples
print("Accuracy:", accuracy)


Accuracy: 0.0


# **Saving input text(Sylheti Language) , labels(normal bangla) and generated translations to csv**

In [ ]:
import pandas as pd

# Create a DataFrame to store translations
translation_df = pd.DataFrame({
    'input_text': test_data['input_text'],  # Assuming 'test_data' contains your test dataset
    'labels': references,
    'translations': generated_translations
})

# Save translations to a CSV file
# Save translations to a CSV file
translation_df.to_excel('/content/drive/MyDrive/Movies/Noakhali_translations_mT5.xlsx', index=False)
